# Load
## 📚 강의 개요 (Overview)

이 강의에서는 텍스트 데이터를 임베딩하고, 벡터 데이터베이스에 로드(Load)하여 효율적으로 저장하고 검색하는 방법을 다룹니다. RAG (Retrieval-Augmented Generation) 시스템에서 어떤 임베딩 모델을 사용할지, 그리고 어떤 벡터 저장소를 활용할지가 검색 및 응답 성능에 중요한 영향을 미칩니다.

이 강의를 통해 다양한 임베딩 모델을 활용하여 텍스트를 벡터로 변환하고, 이를 벡터 데이터베이스에 저장하여 빠르게 검색하는 방법을 배웁니다.

## 목차: 
* [OpenAI 임베딩 모델 활용하기](#openai-임베딩-모델-활용하기)
* [Ollama Embedding 모델 활용하기](#ollama-embedding-모델-활용하기)
* [FAISS로 임베딩 벡터 저장하기](#faiss로-임베딩-벡터-저장하기)
* [Chroma 벡터 DB](#chroma-벡터-db)
* [Qdrant 벡터 DB](#qdrant-벡터-db)

In [ ]:
# 환경변수 설정하기 (.env 파일을 사용하지 않을 경우 여기에 입력해주세요!)
import os

# 환경변수 설정
os.environ["API_KEY"] = "sk-..."

In [1]:
from dotenv import load_dotenv

load_dotenv(override = True)

True

### OpenAI 임베딩 모델 활용하기

`text-embedding-3-small`: OpenAI에서 제공하는 최신 임베딩 모델 중 하나로, 빠르고 가벼운 임베딩을 제공합니다.

문서 임베딩 vs 질의 임베딩
* `embed_documents()` → 여러 개의 문장을 한 번에 임베딩 (문서 검색 등에 활용).
* `embed_query()` → 질의(Query)를 임베딩 (질문-응답 시스템에서 검색할 때 활용).

![](https://arize.com/wp-content/uploads/2022/06/blog-king-queen-embeddings.jpg)

In [2]:
from langchain_openai import OpenAIEmbeddings
embeddings_model = OpenAIEmbeddings(model="text-embedding-3-small")

In [3]:
# 문서(문장) 리스트를 벡터로 변환 (임베딩)
embeddings = embeddings_model.embed_documents(
    [
        "Hi there!",
        "Oh, hello!",
        "What's your name?",
        "My friends call me World",
        "Hello World!"
    ]
)

# 임베딩된 문서 개수와 개별 임베딩 벡터의 차원 출력
len(embeddings), len(embeddings[0])

(5, 1536)

In [4]:
# 질의(Query) 문장을 임베딩 벡터로 변환
embedded_query = embeddings_model.embed_query("What was the name mentioned in the conversation?")
embedded_query[:5] #처음 5개의 값만 출력 

[-0.010680568404495716,
 -0.01018487848341465,
 -0.0019450020045042038,
 0.023096051067113876,
 -0.02682921662926674]

In [5]:
len(embeddings[0])# 개별 문장의 임베딩 차원 확인

1536

### Ollama Embedding 모델 활용하기

In [6]:
# PDF 문서를 로드하여 페이지별로 저장하는 과정

from langchain_community.document_loaders import PyPDFLoader

file_path = (
    "data/arxiv_paper.pdf"
)

# PDF 로더 객체 생성
loader = PyPDFLoader(file_path)

# PDF의 각 페이지를 저장할 리스트 초기화
pages = []

# PDF를 비동기 방식으로 로드하여 페이지별로 저장
async for page in loader.alazy_load():
    pages.append(page)

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# RecursiveCharacterTextSplitter를 사용하여 텍스트 분할 설정
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,       # 하나의 청크 크기를 1000자로 설정
    chunk_overlap=200,     # 청크 간 200자 겹치게 설정 (문맥 유지 목적)
    length_function=len,   # 텍스트 길이를 측정하는 함수 (len 사용)
    is_separator_regex=False  # separator를 정규식이 아닌 단순 문자열로 처리
)

# PDF에서 로드한 데이터를 텍스트 청크로 분할
texts = text_splitter.split_documents(pages)

print(f"{texts[0].metadata}")# 첫 번째 청크의 메타데이터
print(texts[0].page_content)# 첫 번째 청크의 내용 
print("-"*100)
print(f"{texts[1].metadata}")# 두 번째 청크의 메타데이터
print(texts[1].page_content)# 두 번째 청크의 내용

{'source': 'data/arxiv_paper.pdf', 'page': 0}
Uncertainty in Action: Confidence Elicitation in Embodied Agents
Tianjiao Yu, Vedant Shah, Muntasir Wahed, Kiet A. Nguyen, Adheesh Juvekar
Tal August, Ismini Lourentzou
University of Illinois Urbana-Champaign
{ty41,vrshah4,mwahed2,kietan2,adheesh2,taugust,lourent2}@illinois.edu
https://plan-lab.github.io/ece
Abstract
Expressing confidence is challenging for embod-
ied agents navigating dynamic multimodal en-
vironments, where uncertainty arises from both
perception and decision-making processes. We
present the first work investigating embodied con-
fidence elicitation in open-ended multimodal en-
vironments. We introduce Elicitation Policies,
which structure confidence assessment across
inductive, deductive, and abductive reasoning,
along with Execution Policies, which enhance
confidence calibration through scenario reinter-
pretation, action sampling, and hypothetical rea-
soning. Evaluating agents in calibration and fail-
ure prediction t

In [8]:
texts[:5]

[Document(metadata={'source': 'data/arxiv_paper.pdf', 'page': 0}, page_content='Uncertainty in Action: Confidence Elicitation in Embodied Agents\nTianjiao Yu, Vedant Shah, Muntasir Wahed, Kiet A. Nguyen, Adheesh Juvekar\nTal August, Ismini Lourentzou\nUniversity of Illinois Urbana-Champaign\n{ty41,vrshah4,mwahed2,kietan2,adheesh2,taugust,lourent2}@illinois.edu\nhttps://plan-lab.github.io/ece\nAbstract\nExpressing confidence is challenging for embod-\nied agents navigating dynamic multimodal en-\nvironments, where uncertainty arises from both\nperception and decision-making processes. We\npresent the first work investigating embodied con-\nfidence elicitation in open-ended multimodal en-\nvironments. We introduce Elicitation Policies,\nwhich structure confidence assessment across\ninductive, deductive, and abductive reasoning,\nalong with Execution Policies, which enhance\nconfidence calibration through scenario reinter-\npretation, action sampling, and hypothetical rea-\nsoning. Evalua

In [9]:
from langchain_ollama import OllamaEmbeddings

# "bge-m3" 모델을 사용하여 텍스트 임베딩 생성
embeddings_model=OllamaEmbeddings(model="bge-m3")
# 청크된 문서 리스트를 벡터화하여 임베딩 생성
embeddings = embeddings_model.embed_documents([i.page_content for i in texts])

len(embeddings[0])# 생성된 임베딩 벡터의 차원 확인

1024

![](https://imghub.insilicogen.com/media/photos/51034_43550_1241.jpg)

### FAISS로 임베딩 벡터 저장하기

FAISS(Facebook AI Similarity Search)
*  대량의 벡터 데이터를 효율적으로 검색하는 라이브러리
* 문서 검색, 추천 시스템, 이미지 검색 등에서 활용됨.
* 벡터의 유사도를 계산하여 가장 가까운 문서를 빠르게 찾을 수 있음.

FAISS 벡터 저장소 구축
* `FAISS.from_documents()` → 텍스트 청크를 벡터로 변환하여 FAISS에 저장.
* `embedding=embeddings_model` → Ollama의 `bge-m3` 모델을 사용하여 임베딩을 생성.

FAISS 검색 방식
* `similarity_search(query, k=1)` → 가장 유사한 k개의 문서를 검색.
* `similarity_search(query, k=10, filter={"page": 0})` → 특정 조건(예: page=0)에서 유사한 문서 검색.
* `similarity_search_with_score(query, k=10)` → 검색된 문서와 함께 유사도 점수(거리) 출력.

In [ ]:
# %pip install -qU langchain_community faiss-cpu

In [10]:
from langchain_community.vectorstores import FAISS

#  FAISS 벡터 저장소 생성 (OllamaEmbeddings을 활용)
# 앞서 생성한 청크(`texts`)와 임베딩 모델(`embeddings_model`)을 이용하여 벡터 저장소를 구축
vector_store = FAISS.from_documents(
    documents=texts,
    embedding=embeddings_model # Ollama 임베딩 모델
)

# 벡터 저장소 크기 확인
print(f"청크의 수: {len(texts)}") # 총 청크된 문서 개수
print(f"벡터 저장소에 저장된 문서 수: {vector_store.index.ntotal}")# 총 청크된 문서 개수 출력

청크의 수: 100
벡터 저장소에 저장된 문서 수: 100


In [12]:
# 1개의 유사한 문서를 검색 (기본 검색)
results = vector_store.similarity_search(query="Embodied Agent가 뭐야?",k=10)

# 검색된 문서 출력
for doc in results:
    print(f"* {doc.page_content} [{doc.metadata}]")

* Uncertainty in Action: Confidence Elicitation in Embodied Agents
Tianjiao Yu, Vedant Shah, Muntasir Wahed, Kiet A. Nguyen, Adheesh Juvekar
Tal August, Ismini Lourentzou
University of Illinois Urbana-Champaign
{ty41,vrshah4,mwahed2,kietan2,adheesh2,taugust,lourent2}@illinois.edu
https://plan-lab.github.io/ece
Abstract
Expressing confidence is challenging for embod-
ied agents navigating dynamic multimodal en-
vironments, where uncertainty arises from both
perception and decision-making processes. We
present the first work investigating embodied con-
fidence elicitation in open-ended multimodal en-
vironments. We introduce Elicitation Policies,
which structure confidence assessment across
inductive, deductive, and abductive reasoning,
along with Execution Policies, which enhance
confidence calibration through scenario reinter-
pretation, action sampling, and hypothetical rea-
soning. Evaluating agents in calibration and fail-
ure prediction tasks within the Minecraft envi- [{'source': 

In [13]:
# 특정 페이지에 대한 필터링 검색 (page=0인 문서에서 검색)
results = vector_store.similarity_search(query="Embodied Agent가 뭐야?",k=10,filter={"page": 0})

# 필터링된 문서 출력
for doc in results:
    print(f"* {doc.page_content} [{doc.metadata}]")

* Uncertainty in Action: Confidence Elicitation in Embodied Agents
Tianjiao Yu, Vedant Shah, Muntasir Wahed, Kiet A. Nguyen, Adheesh Juvekar
Tal August, Ismini Lourentzou
University of Illinois Urbana-Champaign
{ty41,vrshah4,mwahed2,kietan2,adheesh2,taugust,lourent2}@illinois.edu
https://plan-lab.github.io/ece
Abstract
Expressing confidence is challenging for embod-
ied agents navigating dynamic multimodal en-
vironments, where uncertainty arises from both
perception and decision-making processes. We
present the first work investigating embodied con-
fidence elicitation in open-ended multimodal en-
vironments. We introduce Elicitation Policies,
which structure confidence assessment across
inductive, deductive, and abductive reasoning,
along with Execution Policies, which enhance
confidence calibration through scenario reinter-
pretation, action sampling, and hypothetical rea-
soning. Evaluating agents in calibration and fail-
ure prediction tasks within the Minecraft envi- [{'source': 

In [ ]:
# 유사도 점수를 포함한 검색 (Query와 문서 간 거리 계산)
results = vector_store.similarity_search_with_score(query="Embodied Agent가 뭐야?",k=10)

print("여기서의 score는 query와 문서의 거리를 나타내기 때문에, 낮을수록 유사합니다.\n")

# 검색된 문서와 점수 출력
for doc, score in results:
    print(f"* [유사도={score:3f}] {doc.page_content[:100]} [{doc.metadata}]")
    print("-"*100)

### Chroma 벡터 DB


* ChromaDB는 벡터 데이터베이스로, 텍스트 검색과 추천 시스템 등에 활용됨.
* FAISS는 메모리 내(in-memory)에서 작동하지만, Chroma는 영구 저장(Persistent Storage) 가능.
* Chroma는 쿼리 시 더 다양한 필터링과 조합이 가능함.

**Chroma 벡터 저장소 구축**
* `Chroma(collection_name="test_01", embedding_function=embeddings_model)`→ "test_01"이라는 컬렉션을 생성하고, Ollama 임베딩 모델을 사용하여 벡터 저장.
* `add_documents(documents=texts, ids=ids)`→ 임베딩을 생성한 후 ChromaDB에 저장.

**유사도 검색 방식**
* `similarity_search(query, k=1)` → 가장 유사한 k개의 문서를 검색.
* `similarity_search_with_score(query, k=5)` → 문서와 쿼리 간의 유사도 점수를 함께 반환.


In [ ]:
#%pip install -qU chromadb langchain-chroma

In [14]:
from langchain_chroma import Chroma
from uuid import uuid4

#  Chroma 벡터 저장소 생성
vector_store = Chroma(
    collection_name="test_02",  # 데이터가 저장될 컬렉션 이름
    embedding_function=embeddings_model,  # Ollama 임베딩 모델 사용
)

#  UUID를 활용하여 문서별 고유 ID 생성
ids = [str(uuid4()) for _ in range(len(texts))]

# 벡터 저장소에 문서 추가 (임베딩 자동 생성)
vector_store.add_documents(documents=texts, ids=ids)

['08f1e927-7586-4b72-b522-f5a728e162fd',
 '5fd0ac08-167d-4818-95b9-60863a79ad63',
 '6f10d2dc-8abd-40e5-97c1-4eb59cf80969',
 '5027fbb7-788d-4936-9eef-030e04e93900',
 '4acd9c25-f2b9-4796-8fd5-caec29b55672',
 '16b1114c-1184-49b1-b08f-8fa6d7ed4715',
 '7a3d32e9-0121-4770-a125-7b76db933224',
 'baaca446-fffb-45fa-96f7-d14e186a5fe7',
 'da749c3d-6718-4aa2-a761-0c5714231862',
 'fb0ad5d6-460d-4095-afcb-a0e86c2c8029',
 '9229acab-ad73-4999-910e-ec60e0633167',
 'a02d917d-f1c3-4012-a27f-d71335b31756',
 '32ae1fbf-0b5e-494c-9790-3dc5961ade49',
 '974b2322-f4fb-49e9-a1d9-08ffa4a7c5a7',
 '921de3e4-bee4-4cb4-b3e4-b88948cde0ce',
 'cd7bc759-b09b-4fd8-b925-f12f16a7521f',
 '8308a0d8-f9c2-445c-b98f-93b856ebd5ba',
 'f8573157-6cc7-4156-bbfe-3650e0acf51e',
 '25c136b4-f15d-4c9e-8235-177aaaff2619',
 '2ab6401c-bb7f-4656-b502-ffdc379ebfd7',
 'e6713bab-5bd9-465a-956b-1f97607dd9fc',
 'fc1fd8c6-5b0d-4dc4-b0d5-92e1c3df28cc',
 'a0e8c55f-c72a-4470-8eb6-b1c0369b234f',
 '47df9438-c328-4be9-bd25-731ba907fa52',
 '855b459c-70a5-

In [15]:
# 1개의 유사한 문서를 검색 (기본 검색)
results = vector_store.similarity_search(query="Embodied_agent가 뭐야?", k=1)

# 검색된 문서 출력
for doc in results:
    print(f"* {doc.page_content} [{doc.metadata}]")

* Uncertainty in Action: Confidence Elicitation in Embodied Agents
Tianjiao Yu, Vedant Shah, Muntasir Wahed, Kiet A. Nguyen, Adheesh Juvekar
Tal August, Ismini Lourentzou
University of Illinois Urbana-Champaign
{ty41,vrshah4,mwahed2,kietan2,adheesh2,taugust,lourent2}@illinois.edu
https://plan-lab.github.io/ece
Abstract
Expressing confidence is challenging for embod-
ied agents navigating dynamic multimodal en-
vironments, where uncertainty arises from both
perception and decision-making processes. We
present the first work investigating embodied con-
fidence elicitation in open-ended multimodal en-
vironments. We introduce Elicitation Policies,
which structure confidence assessment across
inductive, deductive, and abductive reasoning,
along with Execution Policies, which enhance
confidence calibration through scenario reinter-
pretation, action sampling, and hypothetical rea-
soning. Evaluating agents in calibration and fail-
ure prediction tasks within the Minecraft envi- [{'page': 0,

In [16]:
# 유사도 점수를 포함한 검색 (Query와 문서 간 거리 계산)
results = vector_store.similarity_search_with_score(query="Embodied_agent가 뭐야?", k=5)

print("여기서의 score는 query와 문서의 거리를 나타내기 때문에, 낮을수록 유사합니다.\n")

# 검색된 문서와 점수 출력
for doc, score in results:
    print(f"* [유사도={score:.3f}] {doc.page_content[:100]} [{doc.metadata}]")  # 상위 100자만 출력
    print("-" * 100)

여기서의 score는 query와 문서의 거리를 나타내기 때문에, 낮을수록 유사합니다.

* [유사도=0.832] Uncertainty in Action: Confidence Elicitation in Embodied Agents
Tianjiao Yu, Vedant Shah, Muntasir  [{'page': 0, 'source': 'data/arxiv_paper.pdf'}]
----------------------------------------------------------------------------------------------------
* [유사도=0.855] Confidence Elicitation in Embodied Agents
Malinin, A. and Gales, M. Uncertainty estimation in autore [{'page': 10, 'source': 'data/arxiv_paper.pdf'}]
----------------------------------------------------------------------------------------------------
* [유사도=0.858] Confidence Elicitation in Embodied Agents
8. Impact Statement
This work advances Embodied AI by intr [{'page': 8, 'source': 'data/arxiv_paper.pdf'}]
----------------------------------------------------------------------------------------------------
* [유사도=0.859] tend to yield improved confidence calibration. For instance,
MineLLM’s ECE achieves 0.32 and 0.30 pa [{'page': 5, 'source': 'data/arxiv_paper.p

### Qdrant 벡터 DB

* Qdrant는 벡터 검색(Vector Search) 및 필터링을 지원하는 벡터 데이터베이스.
* 메모리 내(in-memory) 실행 가능하며, 영구 저장(persistent storage)도 지원.
* FAISS나 Chroma와 다르게, 메타데이터 기반 필터링 기능이 강력함.

**Qdrant 벡터 저장소 구축**
* `QdrantClient(":memory:")` → 인메모리 벡터 저장소 생성.
* `create_collection()` → 1024차원의 벡터를 저장할 수 있는 컬렉션 생성.
* `distance=Distance.COSINE` → 문서 간 코사인 유사도를 계산하여 가장 유사한 문서를 찾음.
  
**Qdrant 검색 기능**
* `similarity_search(query, k=1)` → 가장 유사한 k개의 문서를 검색.
* `similarity_search_with_score(query, k=5)` → 문서와 쿼리 간의 유사도 점수를 함께 반환.
* `similarity_search(query, k=1, filter=...)` → 특정 조건을 만족하는 문서만 검색.

In [ ]:
#%pip install -qU langchain_qdrant

In [17]:
# LangChain에서 Qdrant 벡터 저장소 불러오기
from langchain_qdrant import QdrantVectorStore

# Qdrant 클라이언트 라이브러리 불러오기
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

# Qdrant 클라이언트 생성 (메모리 기반)
# ":memory:" 옵션을 사용하면 휘발성(In-Memory) 데이터베이스로 실행됨.
client = QdrantClient(":memory:")

#  Qdrant 컬렉션 생성
client.create_collection(
    collection_name="test",  # 저장소 이름
    vectors_config=VectorParams(
        size=1024,  # 벡터 차원 수 (사용하는 임베딩 모델에 맞춰야 함)
        distance=Distance.COSINE  # 벡터 간 유사도 측정 방식 (코사인 거리 사용)
    ),
)

# Qdrant 벡터 저장소 객체 생성
vector_store = QdrantVectorStore(
    client=client,  # Qdrant 클라이언트
    collection_name="test",  # 컬렉션 이름
    embedding=embeddings_model  # Ollama 임베딩 모델 사용
)

In [18]:
# 문서 ID를 생성하기 위해 UUID 사용
from uuid import uuid4

# 각 문서에 대해 고유한 ID 생성
ids = [str(uuid4()) for _ in range(len(texts))]

# 벡터 저장소에 문서 추가
vector_store.add_documents(documents=texts, ids=ids)

['da36808d-fec5-4bd3-aa32-5fbe8b3b6e3b',
 '34b26675-c87c-49c4-a57d-db92449800f4',
 '2978d1a9-7afa-43bb-87d2-39b3a8f49d4f',
 '01ab23a7-e09c-4271-a4bd-1f9a82c0e462',
 '1fc0ce8b-65ea-4d1d-a24d-a1d1a8b9bb39',
 '3f694b1c-1097-4fc8-a8b0-ff2d366d93ef',
 'fd6c7712-bd6b-45f7-b1f3-cbd9589bbfcd',
 'fb10eb95-5f84-49f4-b6d9-d0b98005901b',
 '0fa10f84-e13d-49f2-a968-eb8bf0b47516',
 'c3a4f21f-be6f-41ba-8f6f-551e34062aed',
 '66b833f2-6f03-43fa-9f77-663c462a953b',
 '5b73b63f-270c-4f99-b225-1fbedfc99f6d',
 '4180743a-77d1-4a3f-9031-cb310326daa0',
 'b97b63f2-8802-4899-a9f1-c0a37ab84473',
 '4b9ffeb3-2873-499c-b177-d54c1206408d',
 '7aeefedc-2d75-4bed-89d2-2d5bd8c3822d',
 '2faa4fee-b58b-4c15-9b32-83e7786c9111',
 '5ad41bb7-5081-4f48-870f-ac27a6e34d52',
 'a4cfe741-a355-40d8-a649-63c86cae48e6',
 '00c768e4-6f1a-4e11-b9c5-74c49bb1b901',
 'a973d117-34d1-4911-83c3-40542eba90a3',
 '3cca1d72-e76c-44a6-9943-88853a7e03c7',
 'e30ff073-5f14-4b54-b5a7-6fe09846d0db',
 '0815a734-e616-4013-b24f-a61ffdf5750f',
 '71c0c67e-ceff-

In [19]:
# 1개의 유사한 문서를 검색 (기본 검색)
results = vector_store.similarity_search(query="Embodied_agent가 뭐야?", k=1)

# 검색된 문서 출력
for doc in results:
    print(f"* {doc.page_content} [{doc.metadata}]")

* Uncertainty in Action: Confidence Elicitation in Embodied Agents
Tianjiao Yu, Vedant Shah, Muntasir Wahed, Kiet A. Nguyen, Adheesh Juvekar
Tal August, Ismini Lourentzou
University of Illinois Urbana-Champaign
{ty41,vrshah4,mwahed2,kietan2,adheesh2,taugust,lourent2}@illinois.edu
https://plan-lab.github.io/ece
Abstract
Expressing confidence is challenging for embod-
ied agents navigating dynamic multimodal en-
vironments, where uncertainty arises from both
perception and decision-making processes. We
present the first work investigating embodied con-
fidence elicitation in open-ended multimodal en-
vironments. We introduce Elicitation Policies,
which structure confidence assessment across
inductive, deductive, and abductive reasoning,
along with Execution Policies, which enhance
confidence calibration through scenario reinter-
pretation, action sampling, and hypothetical rea-
soning. Evaluating agents in calibration and fail-
ure prediction tasks within the Minecraft envi- [{'source': 

In [20]:
#  특정 필터링 조건을 적용한 검색
from qdrant_client.http import models

results = vector_store.similarity_search(
    query="Embodied_agent가 뭐야?",
    k=1,
    filter=models.Filter(must=[models.FieldCondition(
        key="metadata.page",  # 특정 필드(예: page) 기반 필터링
        match=models.MatchValue(value=1),  # page=1인 문서만 검색
    )])
)

# 필터링된 문서 출력
for doc in results:
    print(f"* {doc.page_content} [{doc.metadata}]")

* Confidence Elicitation in Embodied Agents
understanding high-level tasks or goals ( e.g., “I am 70%
confident I craft a wooden table”). 3) Interactive depen-
dencies, where the agent’s actions directly influence the
environment, which in turn affects subsequent decisions,
requiring ongoing adjustments to confidence estimates as
tasks progress. 4) Finally, while state-of-the-art embodied
agents leverage proprietary Large Language Models (LLMs)
and Vision-Language Models (VLMs) for their strong mul-
timodal understanding and reasoning capabilities (Wang
et al., 2023a; Qin et al., 2024; Zhu et al., 2023), these of-
ten lack access to internal token likelihoods or probabilistic
outputs, making traditional confidence estimation methods
ineffective (Kumar et al., 2023; Chen et al., 2024b).
To address these challenges, we present the first system-
atic approach that enables LLM/VLM-powered embodied
agents to assess and articulate their confidence across multi- [{'source': 'data/arxiv_paper.

In [ ]:
# 유사도 점수를 포함한 검색 (Query와 문서 간 거리 계산)
results = vector_store.similarity_search_with_score(query="Embodied_agent가 뭐야?", k=5)

print("여기서의 score는 query와 문서의 유사한 정도를 나타내기 때문에, 높을수록 유사합니다.\n")

# 검색된 문서와 점수 출력
for doc, score in results:
    print(f"* [유사도={score:.3f}] {doc.page_content[:1000]} [{doc.metadata}]")  # 첫 1000자 출력
    print("-" * 100)

여기서의 score는 query와 문서의 거리를 나타내기 때문에, 낮을수록 유사합니다.

* [유사도=0.584] Uncertainty in Action: Confidence Elicitation in Embodied Agents
Tianjiao Yu, Vedant Shah, Muntasir Wahed, Kiet A. Nguyen, Adheesh Juvekar
Tal August, Ismini Lourentzou
University of Illinois Urbana-Champaign
{ty41,vrshah4,mwahed2,kietan2,adheesh2,taugust,lourent2}@illinois.edu
https://plan-lab.github.io/ece
Abstract
Expressing confidence is challenging for embod-
ied agents navigating dynamic multimodal en-
vironments, where uncertainty arises from both
perception and decision-making processes. We
present the first work investigating embodied con-
fidence elicitation in open-ended multimodal en-
vironments. We introduce Elicitation Policies,
which structure confidence assessment across
inductive, deductive, and abductive reasoning,
along with Execution Policies, which enhance
confidence calibration through scenario reinter-
pretation, action sampling, and hypothetical rea-
soning. Evaluating agents in calibration and fail